# ChatGPT를 API에서 이용하기

## 입력과 출력의 길이 제한 및 요금에 영향을 주는 토큰


In [1]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# OpenAI에서 만든 토큰화(Tokenization) 라이브러리 불러오기
# GPT 모델이 문장을 어떻게 쪼개는지 그대로 재현할 수 있다
import tiktoken

# 토큰 개수를 알고 싶은 문장을 문자열(String)로 저장
# 아직 이 상태에서는 그냥 사람이 읽는 문장일 뿐이다
text = "It’s easy to make something cool with LLMs, but very hard to make something production-ready with them."


# 사용할 GPT 모델에 맞는 토크나이저 규칙을 불러온다
# 모델마다 토큰을 쪼개는 방식이 다르기 때문에 모델명을 반드시 지정해야 한다
# 여기서는 gpt-4.1-mini 모델 기준 토큰 규칙을 사용한다
encoding=tiktoken.encoding_for_model("gpt-4.1-mini")

# 위에서 불러온 토큰 규칙을 이용해
# 사람이 읽는 문장(text)을 → 모델이 이해하는 숫자 토큰 리스트로 변환한다
# 예: "Hello" → [15496]
tokens=encoding.encode(text)


# tokens는 [숫자, 숫자, 숫자, ...] 형태의 리스트
# len(tokens)는 이 문장이 몇 개의 토큰으로 구성되어 있는지를 의미한다
# 즉, GPT 모델이 볼 때 이 문장의 길이
print(len(tokens))

23


In [3]:

# 토큰 개수를 알고 싶은 문장을 문자열(String)로 저장한다
# 이 문장은 사람이 읽는 '한글 문장'이다
# GPT 모델은 이 문장을 그대로 이해하지 못하고,
# 반드시 토큰(token)이라는 작은 단위로 쪼개서 처리한다
text = "LLM을 사용해서 멋져 보이는 것을 만들기는 쉽지만, 프로덕션 수준으로 만들어 내기는 매우 어렵다."


# 사용할 GPT 모델 이름을 지정해서,
# 해당 모델이 사용하는 토큰 분해(Tokenizer) 규칙을 불러온다
# 모델마다 토큰을 나누는 기준이 다르기 때문에 반드시 모델명을 정확히 써야 한다
# 여기서는 gpt-4.1-mini 모델 기준 토큰 규칙을 사용한다
encoding=tiktoken.encoding_for_model("gpt-4.1-mini")

# 위에서 불러온 토큰 규칙을 사용해서
# 사람이 읽는 한글 문장(text)을
# GPT 모델이 이해할 수 있는 숫자 토큰들의 리스트로 변환한다
# 예: "LLM" → 하나 또는 여러 개의 숫자 토큰으로 변환될 수 있다
tokens=encoding.encode(text)

# tokens는 [숫자, 숫자, 숫자, ...] 형태의 리스트이다
# 리스트 안의 숫자 하나가 곧 '토큰 1개'를 의미한다
# len(tokens)는 이 리스트에 들어 있는 숫자의 개수
# 즉, 이 한글 문장이 GPT 모델 기준으로 몇 개의 토큰을 차지하는지를 의미한다
# 이 값은 API 비용 계산과 컨텍스트 길이 제한에 직접적인 영향을 준다
print(len(tokens))

28


## 3-4 Chat Completions API를 접하는 환경 준비
OpenAI API 키 준비

In [4]:
# 운영체제(OS)와 관련된 기능을 사용하기 위한 파이썬 기본 라이브러리
# 환경 변수(Environment Variable)를 읽어올 때 사용한다
import os


# .env 파일에 저장된 환경 변수를 불러오기 위한 라이브러리
# API 키 같은 민감한 정보를 코드에 직접 쓰지 않게 해준다
from dotenv import load_dotenv

# 현재 프로젝트 폴더에 있는 .env 파일을 읽어서
# 그 안에 적혀 있는 값들을 "환경 변수"로 등록한다
# 이 코드를 실행한 이후부터 os.getenv()로 값을 읽을 수 있다
load_dotenv()

# 운영체제에 등록된 환경 변수 중에서
# 이름이 "OPENAI_API_KEY"인 값을 가져온다
# .env 파일 안에 아래처럼 적혀 있어야 한다
# OPENAI_API_KEY=sk-xxxxxxxxxxxx
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

## 3-5 Chat Completions API 사용해 보기
OpenAI 라이브러리

In [5]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Chat Completions API 호출

In [6]:
# OpenAI API를 파이썬에서 사용하기 위한 OpenAI 클래스 불러오기
# 이 클래스를 통해 GPT 모델과 통신할 수 있다
from openai import OpenAI


# OpenAI 서버와 통신하기 위한 클라이언트 객체 생성
# 이때 환경 변수에 저장된 OPENAI_API_KEY를 자동으로 읽어 인증한다
client=OpenAI()


# GPT 모델에게 대화를 보내고 응답을 받아오는 부분
response=client.chat.completions.create(
    # 사용할 GPT 모델 이름 지정
    # gpt-4.1-mini는 빠르고 비용이 저렴한 GPT-4 계열 모델
    model="gpt-4.1-mini",
    # GPT에게 전달할 대화 내용
    # messages는 "대화 기록 전체"를 리스트 형태로 전달한다
    messages=[
        # system 역할:
        # GPT의 성격, 말투, 행동 규칙을 정의
        # "너는 친절한 도우미다"라는 기본 설정
        {"role":"system","content":"You are a helpful assistant."},
         # user 역할:
        # 실제 사용자가 입력한 질문 또는 메시지
        {"role":"user","content":"Hello I'm John"}
    ]
)


# -------------------------------------------
# 1️⃣ GPT 응답 객체 전체를 그대로 출력
# -------------------------------------------
# response는 단순한 문자열이 아니라
# 모델 정보, 응답 내용, 토큰 사용량 등이 모두 들어 있는 객체
# 구조를 처음 확인할 때 주로 사용
print(response)

ChatCompletion(id='chatcmpl-CyAfsMNLDSa1ZIsOJbOjmgRFKuwWZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello John! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1768456748, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=10, prompt_tokens=20, total_tokens=30, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [7]:

# -------------------------------------------
# 2️⃣ GPT가 실제로 말한 답변만 출력
# -------------------------------------------
# response.choices:
#   GPT가 생성한 답변 후보들의 리스트
# choices[0]:
#   첫 번째(보통 유일한) 답변
# message.content:
#   GPT가 생성한 실제 텍스트 응답
print(response.choices[0].message.content)


Hello John! How can I assist you today?


In [16]:
print(response.choices[0].message.content)

Yes, you mentioned that your name is John. How can I assist you today?


In [8]:
# -------------------------------------------
# 3️⃣ 응답 객체를 JSON 형태로 예쁘게 출력
# -------------------------------------------
# model_dump_json():
#   response 객체를 JSON 문자열로 변환
# indent=2:
#   들여쓰기를 적용해 사람이 읽기 쉽게 출력
# API 응답 구조를 정확히 이해하거나
# 로그 저장, 디버깅할 때 매우 유용
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CyAfsMNLDSa1ZIsOJbOjmgRFKuwWZ",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello John! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1768456748,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 20,
    "total_tokens": 30,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


## 대화 이력을 바탕으로 한 응답 얻기

In [9]:
# GPT 모델에게 대화 전체를 보내고,
# 그 다음에 올 GPT의 응답을 생성하도록 요청한다
response=client.chat.completions.create(

    # 사용할 GPT 모델을 지정한다
    # gpt-4.1-mini는 속도가 빠르고 비용이 비교적 저렴한 모델이다
    model='gpt-4.1-mini',

    # messages는 GPT에게 전달하는 "대화 기록 전체"이다
    # GPT는 이 리스트를 처음부터 끝까지 읽고 다음 응답을 만든다
    messages=[

         # system 역할:
        # GPT의 성격, 말투, 행동 규칙을 정의한다
        # 이 대화 전체에서 GPT는 "친절한 도우미"로 행동한다
        {"role":"system","content":"You are a helpful assistant"},
        # user 역할:
        # 사용자가 처음으로 한 말
        # 여기서 사용자는 자신의 이름이 John이라고 밝히고 있다
        {"role":"user","content":"Hello! I'm John."},

        # assistant 역할:
        # 이전에 GPT가 했던 대답을 그대로 적어준다
        # 이렇게 해야 GPT가 "이전에 무엇을 말했는지"를 알 수 있다
        {"role":"assistant","content":"Hello John! How can I assist you today?"},

        # user 역할:
        # 사용자가 새로 던진 질문
        # GPT는 위의 모든 대화 내용을 보고 이 질문에 답하게 된다
        {"role":"user","content":"Do you know my name?"}
    ]
)



# response 객체 전체를 JSON 형태로 출력한다
# indent=2는 들여쓰기를 적용해서 사람이 읽기 쉽게 만든다
# 이 출력에는 GPT의 답변, 모델 정보, 토큰 사용량 등이 모두 포함된다
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CyAft4BQV9ZsmF3A8YtukB9VS0afG",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Yes, you mentioned that your name is John. How can I help you today, John?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1768456749,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 45,
    "total_tokens": 64,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


## 응답을 스트리밍으로 받기

In [ ]:
# GPT 모델과 통신하기 위해 OpenAI 클라이언트를 사용한다
# client 객체는 이미 생성되어 있다고 가정한다
response=client.chat.completions.create(
    # 사용할 GPT 모델을 지정한다
    # gpt-4.1-mini는 빠르고 비용이 저렴한 GPT-4 계열 모델이다
    model="gpt-4.1-mini",

     # GPT에게 전달할 대화 기록 전체
    # GPT는 이 리스트를 처음부터 끝까지 읽고
    # 다음에 할 말을 토큰 단위로 생성한다
    messages=[

        # system 역할:
        # GPT의 성격과 행동 방식을 정의한다
        # 이 대화에서 GPT는 "친절한 도우미"로 행동한다
        {"role":"system","content":"You are a helpful assistant"},

        # user 역할:
        # 사용자가 GPT에게 실제로 입력한 질문
        {"role":"user","content":"Hello! I'm John"}
    ],

    # stream=True 옵션:
    # GPT가 답변을 전부 만든 뒤 한 번에 보내는 것이 아니라
    # 토큰(글자 조각)을 생성하는 즉시
    # 조금씩 나누어 실시간으로 보내도록 설정한다
    stream=True
)


# response는 하나의 결과 객체가 아니라
# 여러 개의 응답 조각(chunk)을 순서대로 받을 수 있는 반복 객체이다
# 따라서 for문으로 하나씩 꺼내서 처리한다
for chunk in response:
    # GPT는 여러 답변 후보를 만들 수 있기 때문에
    # choices는 리스트 형태로 들어 있다
    # 일반적으로 첫 번째(choice[0]) 응답만 사용한다
    choice=chunk.choices[0]
    # finish_reason이 None이면
    # GPT가 아직 답변을 생성 중이라는 의미이다
    # 답변이 끝나면 finish_reason에 "stop" 같은 값이 들어간다
    if choice.finish_reason is None:
        # delta는 이번 chunk에서 새로 추가된 내용만 담고 있다
        # 전체 문장이 아니라, 방금 생성된 글자 조각이다
        # 이 값을 출력하면 GPT가 타이핑하듯이 글자가 나타난다
        print(choice.delta.content)
        print(type(choice.delta.content))
        


<class 'str'>
Hello
<class 'str'>
 John
<class 'str'>
!
<class 'str'>
 How
<class 'str'>
 can
<class 'str'>
 I
<class 'str'>
 assist
<class 'str'>
 you
<class 'str'>
 today
<class 'str'>
?
<class 'str'>


## 칼럼）Completions API

In [11]:
# OpenAI API를 사용하기 위해 OpenAI 클래스를 불러온다
# 이 클래스는 GPT 모델과 통신하기 위한 핵심 도구이다
from openai import OpenAI



# OpenAI 서버와 통신할 수 있는 클라이언트 객체를 생성한다
# 이때 운영체제 환경 변수에 저장된 OPENAI_API_KEY를 자동으로 읽어온다
# (즉, API 키를 코드에 직접 쓰지 않아도 된다)
client=OpenAI()

# GPT 모델에게 메시지를 보내고,
# 그에 대한 GPT의 응답을 받아서 response 변수에 저장한다
response=client.chat.completions.create(
    # 사용할 GPT 모델을 지정한다
    # gpt-4.1-mini는 속도가 빠르고 비용이 저렴한 GPT-4 계열 모델이다
    model="gpt-4.1-mini",
    # GPT에게 전달할 대화 내용
    # messages는 항상 리스트 형태이며,
    # GPT는 이 리스트를 대화 기록으로 인식한다
     # role이 "user"인 메시지:
    # 실제 사용자가 GPT에게 입력한 문장
    messages=[
        {"role":"user","content":"Hello! I'm John."}
    ]
)
# response 객체 전체를 JSON 형태로 변환하여 출력한다
# model_dump_json()은 response 안에 들어 있는 모든 정보를 보여준다
# indent=2는 들여쓰기를 적용해 사람이 읽기 쉽게 만든다
# 이 출력에는 GPT의 답변, 사용된 모델, 토큰 사용량 등이 포함된다
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CyAfvufpOZKm2zO7SB5FBbJUwNUyl",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello John! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1768456751,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 12,
    "total_tokens": 22,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [12]:
# -------------------------------
# GPT에게 전달할 "대화 기록"을 미리 만든다
# messages는 리스트(list) 형태이며,
# GPT는 이 리스트를 처음부터 끝까지 읽고
# 다음에 할 말을 생성한다
# -------------------------------
messages=[
    # user 역할:
    # 실제 사용자가 처음으로 한 말
    # 사용자는 자신의 이름이 John이라고 말하고 있다
    {"role":"user","content":"Hello! I'm John."},
    # assistant 역할:
    # 이전에 GPT가 했던 대답을 그대로 넣어준다
    # 이렇게 해야 GPT가 "이전에 무엇을 말했는지"를 알 수 있다
    {"role":"assistant","content":"Nice to meet you, John!"},
     # user 역할:
    # 사용자가 이어서 한 새로운 질문
    # GPT는 위의 모든 대화 내용을 보고 이 질문에 답한다
    {"role":"user","content":"Do you know my name?"}
]


# ----------------------------------------
# GPT 모델에게 위에서 만든 대화 기록을 보내고
# 그 다음에 올 GPT의 응답을 생성하도록 요청한다
# ----------------------------------------
response=client.chat.completions.create(
    # 사용할 GPT 모델을 지정한다
    # gpt-4.1-mini는 빠르고 비용이 비교적 저렴한 GPT-4 계열 모델이다
    model='gpt-4.1-mini',
    # 위에서 만든 messages 리스트를 그대로 전달한다
    # 이 리스트가 곧 GPT가 참고하는 "대화의 기억"이다
    messages=messages
)


# ----------------------------------------
# GPT의 응답 전체를 JSON 형태로 출력한다
# indent=2는 들여쓰기를 적용해
# 사람이 읽기 쉽게 만들어 준다
# 이 출력에는 GPT의 실제 답변, 모델 정보,
# 토큰 사용량 등이 모두 포함된다
# ----------------------------------------
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CyAfvbZkAFHxrIzRDu3uIHNQqaVW2",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Yes, you mentioned that your name is John. How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1768456751,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 33,
    "total_tokens": 50,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
